In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import *

In [3]:
textfile = sc.textFile("file:///home/hduser/export_dataframe.csv")
print(textfile)


file:///home/hduser/export_dataframe.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [4]:
df = spark.read.csv("file:///home/hduser/export_dataframe.csv" , inferSchema=True , header=True)
print(df.show())
print(df.printSchema())

+--------------------+-------------------+-------------------+-------------------+-------------------+
|            username|          idle_time|       working_hour|         start_time|           End_time|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|  sahil24c@gmail.com|2019-10-24 12:40:00|2019-10-24 23:55:00|2019-10-24 05:30:01|2019-10-25 05:25:01|
|  yathink3@gmail.com|2019-10-24 12:30:00|2019-10-24 23:55:01|2019-10-24 05:30:01|2019-10-25 05:25:02|
|magadum.iranna@gm...|2019-10-24 13:35:00|2019-10-24 23:55:01|2019-10-24 05:30:01|2019-10-25 05:25:02|
|  shelkeva@gmail.com|2019-10-24 00:40:00|2019-10-24 09:40:01|2019-10-24 08:45:01|2019-10-24 18:25:02|
|puruissimple@gmai...|2019-10-24 01:50:00|2019-10-24 10:49:59|2019-10-24 08:50:02|2019-10-24 19:40:01|
|sangita.awaghad19...|2019-10-24 01:05:00|2019-10-24 10:35:00|2019-10-24 08:50:01|2019-10-24 19:25:01|
|vaishusawant143@g...|2019-10-24 00:05:00|2019-10-24 10:50:01|2019-10-24 

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
df.describe('working_hour').show()

In [ ]:
from numpy import mean
df.select('username','working_hour').show()
df.select(mean(df("working_hour"))).show()

In [ ]:
early_commers = df.filter(df.idle_time>="2019-10-24 09:30:00")

In [ ]:
df.dtypes

In [ ]:
df.select(mean(df("working_hour"))).show()

In [ ]:
df.select(mean("working_hour")).show()

In [ ]:
df.select(mean['working_hour']).show()

In [6]:
#df.groupBy('username').agg({'idle_time': 'mean'}).show()
import datetime
from pyspark.sql.functions import col, lit, avg 
new_working_hour = []
converted_seconds_sum = 0
for row in df.collect():
    date_time = datetime.datetime.strptime(row['working_hour'], "%Y-%m-%d %H:%M:%S")
    converted_seconds = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    new_working_hour.append((row['username'], converted_seconds))
    converted_seconds_sum = converted_seconds_sum + converted_seconds
    #print(new_working_hour) 
avg_seconds = converted_seconds_sum/88
print(avg_seconds)
new_column = sqlContext.createDataFrame(new_working_hour, ('username',"seconds"))
#new_column.filter(new_column['seconds'] > avg_seconds).show()
#joined_dataframe = (new_column.join(new_column, col("username")==col("username"),"leftouter").drop("username"))


new_column.filter(new_column.seconds >= avg_seconds).show()

#average_object = new_column.agg(avg(col("seconds"))
#average_object = new_column.agg(avg(col("seconds"))).show()
#new_column.filter(new_column['seconds'] > ).agg(avg(col("seconds"))).show()

36044
+--------------------+-------+
|            username|seconds|
+--------------------+-------+
|  sahil24c@gmail.com|  86100|
|  yathink3@gmail.com|  86101|
|magadum.iranna@gm...|  86101|
|puruissimple@gmai...|  38999|
|sangita.awaghad19...|  38100|
|vaishusawant143@g...|  39001|
|     you@example.com|  38400|
|vishnu23kumar@gma...|  39301|
|ashutoshrit64@gma...|  43200|
|akshaybavalekar10...|  37200|
|khairnarswapna99@...|  41400|
|kukadeshilpaa7m95...|  37200|
|sarikabarge111@gm...|  36300|
|narsimharaj.kasu0...|  36600|
|antonyalexcm@gmai...|  37501|
|jitupatil937@gmai...|  37500|
|akshaypatwari24@g...|  36900|
|aheteshams007@gma...|  39000|
|sargampandey27oct...|  36899|
|ayush.saraf47@gma...|  36601|
+--------------------+-------+
only showing top 20 rows

